## Prepare data reader for a given chain and date

In [1]:
from op_analytics.coreutils.duckdb_inmem import init_client
from op_analytics.coreutils.partitioned import DataReader, DataLocation
from op_analytics.datapipeline.etl.intermediate.construct import construct_data_readers

from op_analytics.datapipeline.etl.intermediate.udfs import create_duckdb_macros


# Define the input data range.
read_batches: list[DataReader] = construct_data_readers(
    chains=["op"],
    range_spec="@20241118:+1",
    read_from=DataLocation.GCS
)


# Select input for one date and build the intermediate model inputs.
batch = read_batches[0]


duckdb_client = init_client()
create_duckdb_macros(duckdb_client)


2024-11-22 20:53:56 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=26
2024-11-22 20:53:56 [debug    ] loaded vault: 16 items         filename=vault.py lineno=74
2024-11-22 20:53:57 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno=38
2024-11-22 20:53:57 [info     ] prepared 1 input batches.      filename=reader.py lineno=147


## Run the model

This automatically registers the model outputs as duckdb tables.

In [2]:
from op_analytics.datapipeline.etl.intermediate.registry import REGISTERED_INTERMEDIATE_MODELS, load_model_definitions
from op_analytics.datapipeline.etl.intermediate.testutils import execute_model_in_memory

load_model_definitions()
execute_model_in_memory(
    duckdb_client=duckdb_client,
    model=REGISTERED_INTERMEDIATE_MODELS["creation_traces"],
    data_reader=batch
)

# The duckdb database will have the following:
#   - input tables
#   - views used by the model
#   - model outputs
# 
# You can use duckdb to inspect any of the above results.
duckdb_client.sql("SHOW TABLES")

2024-11-22 20:53:57 [info     ] Rendering query                filename=querybuilder.py lineno=51 model=daily_address_summary template=transaction_fees
2024-11-22 20:53:57 [warning  ] Rendered SQL template does not match expectation. Please update: /Users/chuxinhuang/work/op-analytics/src/op_analytics/datapipeline/etl/intermediate/rendered/transaction_fees.rendered.sql filename=querybuilder.py lineno=65 model=daily_address_summary
2024-11-22 20:53:57 [info     ] Rendering query                filename=querybuilder.py lineno=51 model=daily_address_summary template=daily_address_summary
2024-11-22 20:53:57 [info     ] Rendering query                filename=querybuilder.py lineno=51 model=creation_traces template=creation_traces
2024-11-22 20:53:57 [warning  ] Rendered SQL template does not match expectation. Please update: /Users/chuxinhuang/work/op-analytics/src/op_analytics/datapipeline/etl/intermediate/rendered/creation_traces.rendered.sql filename=querybuilder.py lineno=65 model=cre

┌────────────────────┐
│        name        │
│      varchar       │
├────────────────────┤
│ creation_traces    │
│ creation_traces_v1 │
│ traces             │
│ transactions       │
└────────────────────┘

## Verify model results

In [3]:
duckdb_client.sql("SELECT * FROM creation_traces_v1 LIMIT 10")

┌─────────┬──────────┬─────────┬────────────┬───────────────────┬──────────────────────┬────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────┬──────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬────────────────────────────────────────────┬──────────┬────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬───────────┬─────────────┬──────────┬──────────┬───────────┬───────────────┬─────────┬────────┬──────────────────────┬─────────────────┐
│ network │ chain_id │  chain  │     dt     │ created_timestamp │ created_block_number │                         created_block_hash                         │                          crea

### Check the data output size

In [4]:
duckdb_client.sql("SELECT COUNT(*) FROM creation_traces_v1")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│         6484 │
└──────────────┘

In [5]:
duckdb_client.sql("SELECT input FROM traces LIMIT 10")


┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                                                                                                                                                                   input                                                                                                                                                                                                    │
│                                                                                                                                                                                                  varchar

### You can also convert the results to dataframes to inspect them in more familiar ways

In [6]:
duckdb_client.sql("SELECT * FROM creation_traces_v1 LIMIT 10").pl().head()

network,chain_id,chain,dt,created_timestamp,created_block_number,created_block_hash,created_tx_hash,created_tx_index,trace_creator_address,created_tx_from,contract_address,created_tx_to,value_64,value_lossless,code,output,trace_type,call_type,reward_type,gas,gas_used,subtraces,trace_address,error,status,created_tx_method_id,code_bytelength
str,i32,str,str,u32,i64,str,str,i64,str,str,str,str,i64,str,str,str,str,str,str,i64,i64,i64,str,str,i64,str,f64
"""mainnet""",10,"""op""","""2024-11-18""",1731902979,128152101,"""0x44d3a29d2a697b22c845d96573a7…","""0xd857f4101f21eebb978da0d16a2c…",1,"""0x9ec1c3dcf667f2035fb4cd2eb42a…","""0xab8d807a29560ee2f2bf97c75201…","""0x07a242364b2a109ddcca429a9ccf…","""0x9ec1c3dcf667f2035fb4cd2eb42a…",0,"""0""","""0x3d602d80600a3d3981f3363d3d37…","""0x363d3d373d3d3d363d739ec1c3dc…","""create2""","""""","""""",26041689,9031,0,"""40""","""""",1,"""0xb1ae2ed1""",55.0
"""mainnet""",10,"""op""","""2024-11-18""",1731902979,128152101,"""0x44d3a29d2a697b22c845d96573a7…","""0xd857f4101f21eebb978da0d16a2c…",1,"""0x9ec1c3dcf667f2035fb4cd2eb42a…","""0xab8d807a29560ee2f2bf97c75201…","""0x2186affb22fb08c0af667ef839ee…","""0x9ec1c3dcf667f2035fb4cd2eb42a…",0,"""0""","""0x3d602d80600a3d3981f3363d3d37…","""0x363d3d373d3d3d363d739ec1c3dc…","""create2""","""""","""""",25875756,9031,0,"""42""","""""",1,"""0xb1ae2ed1""",55.0
"""mainnet""",10,"""op""","""2024-11-18""",1731902979,128152101,"""0x44d3a29d2a697b22c845d96573a7…","""0xd857f4101f21eebb978da0d16a2c…",1,"""0x9ec1c3dcf667f2035fb4cd2eb42a…","""0xab8d807a29560ee2f2bf97c75201…","""0x0783dfcc9bd74923eec4641f3b84…","""0x9ec1c3dcf667f2035fb4cd2eb42a…",0,"""0""","""0x3d602d80600a3d3981f3363d3d37…","""0x363d3d373d3d3d363d739ec1c3dc…","""create2""","""""","""""",25709823,9031,0,"""44""","""""",1,"""0xb1ae2ed1""",55.0
"""mainnet""",10,"""op""","""2024-11-18""",1731902979,128152101,"""0x44d3a29d2a697b22c845d96573a7…","""0xd857f4101f21eebb978da0d16a2c…",1,"""0x9ec1c3dcf667f2035fb4cd2eb42a…","""0xab8d807a29560ee2f2bf97c75201…","""0x42651c4ca5ee9465dc124a64ae50…","""0x9ec1c3dcf667f2035fb4cd2eb42a…",0,"""0""","""0x3d602d80600a3d3981f3363d3d37…","""0x363d3d373d3d3d363d739ec1c3dc…","""create2""","""""","""""",25543894,9031,0,"""46""","""""",1,"""0xb1ae2ed1""",55.0
"""mainnet""",10,"""op""","""2024-11-18""",1731902979,128152101,"""0x44d3a29d2a697b22c845d96573a7…","""0xd857f4101f21eebb978da0d16a2c…",1,"""0x9ec1c3dcf667f2035fb4cd2eb42a…","""0xab8d807a29560ee2f2bf97c75201…","""0x505f87963a6ae895e027313e3d77…","""0x9ec1c3dcf667f2035fb4cd2eb42a…",0,"""0""","""0x3d602d80600a3d3981f3363d3d37…","""0x363d3d373d3d3d363d739ec1c3dc…","""create2""","""""","""""",25377961,9031,0,"""48""","""""",1,"""0xb1ae2ed1""",55.0


### Get table schema

In [7]:
duckdb_client.sql("DESCRIBE creation_traces_v1")


┌───────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│      column_name      │ column_type │  null   │   key   │ default │  extra  │
│        varchar        │   varchar   │ varchar │ varchar │ varchar │ varchar │
├───────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ network               │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ chain_id              │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ chain                 │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ dt                    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ created_timestamp     │ UINTEGER    │ YES     │ NULL    │ NULL    │ NULL    │
│ created_block_number  │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ created_block_hash    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ created_tx_hash       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ created_tx_index      │ BIGINT      │ 